# Demo notebook for accessing Harmonized Aboveground Global Biomass data on Azure

This notebook provides an example of accessing Harmonized Aboveground Global Biomass (HAGB) data from blob storage on Azure, including (1) looking at the directory where data is stored, (2) load the cloud optimized geotiff for a aprticular year, and (3) doing some simple processing and plotting of the data.

HAGB data are stored in the XYZ data center, so this notebook will run most efficiently on Azure compute located in XYZ. We recommend that substantial computation depending on MODIS data also be situated in XYZ. If you are using MTBS data for environmental science applications, consider applying for an AI for Earth grant to support your compute requirements.

The HAGB dataset provides temporally consistent and harmonized global maps of aboveground and belowground biomass carbon density for the year 2010 at a 300-m spatial resolution. The aboveground biomass map integrates land-cover specific, remotely sensed maps of woody, grassland, cropland, and tundra biomass. Input maps were amassed from the published literature and, where necessary, updated to cover the focal extent or time period. The belowground biomass map similarly integrates matching maps derived from each aboveground biomass map and land-cover specific empirical models. Aboveground and belowground maps were then integrated separately using ancillary maps of percent tree cover and landcover and a rule-based decision tree. Maps reporting the accumulated uncertainty of pixel-level estimates are also provided.

In [2]:
import fsspec
import xarray as xr
import matplotlib.pyplot as plt 

In [3]:
# view the directory of available data 
account_name = 'carbonplan'  # change to your institution! 
container = 'carbonplan-data'  # change! 

fs = fsspec.get_filesystem_class('az')(account_name=account_name)
fs.ls(path=f'{container}/raw/2010-harmonized-biomass/global/300m/')

['carbonplan-data/raw/2010-harmonized-biomass/global/300m/aboveground.tif',
 'carbonplan-data/raw/2010-harmonized-biomass/global/300m/aboveground_uncertainty.tif',
 'carbonplan-data/raw/2010-harmonized-biomass/global/300m/belowground.tif',
 'carbonplan-data/raw/2010-harmonized-biomass/global/300m/belowground_uncertainty.tif']

In [5]:
# load data for above ground biomass
da = xr.open_rasterio(f'https://{account_name}.blob.core.windows.net/{container}/raw/2010-harmonized-biomass/global/300m/aboveground.tif', 
                      chunks=dict(x=2560, y=2560)) # a trick to use dask and chuck the data to get better performance 

# transform our data array to dataset by selecting the only data variable band 
# rename variable to something useful 
ds = da.to_dataset(dim='band').rename({1: 'biomass'})
ds

<xarray.Dataset>
Dimensions:  (x: 129600, y: 52201)
Coordinates:
  * y        (y) float64 84.0 84.0 83.99 83.99 ... -60.99 -61.0 -61.0 -61.0
  * x        (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0 180.0
Data variables:
    biomass  (y, x) uint16 dask.array<chunksize=(2560, 2560), meta=np.ndarray>
Attributes:
    transform:           (0.00277777777777778, 0.0, -180.00000000000006, 0.0,...
    crs:                 +init=epsg:4326
    res:                 (0.00277777777777778, 0.00277777777777778)
    is_tiled:            1
    nodatavals:          (nan,)
    scales:              (1.0,)
    offsets:             (0.0,)
    AREA_OR_POINT:       Area
    DataType:            Generic
    OVR_RESAMPLING_ALG:  NEAREST

In [6]:
# coarsen by 100x on each dimension to get to a reasonable plotting size 
factor = 100
coarse = ds.biomass.coarsen(dim={'x': factor, 'y': factor}, boundary='trim').mean().compute()

In [9]:
plt.figure(figsize=(10,6))
coarse.plot()
plt.show()
plt.close()